# Project Minerva
## Notebook: preprocessing input data for the Global Risks Interconnections

The GDELT [https://www.gdeltproject.org/] dataset is hosted in Google BigQuery database: `gdelt-bq:gdeltv2` [https://bigquery.cloud.google.com/dataset/gdelt-bq:gdeltv2].

We create a table for each of the Global Risks we analyzed and use this notebook to process this input data and obtain the intensity of their interconnections. The queries used in BigQuery to create the tables are documented here: https://github.com/libre-ai/minerva/blob/master/doc/global_risks_bigquery_queries.md


In [7]:
import google.datalab.bigquery as bq
import pandas as pd
from itertools import product

### List of BigQuery tables: 

In [12]:
risk_tables = ["risk_001_asset_bubbles_in_a_major_economy",
               "risk_002_deflation_in_a_major_economy",
               "risk_003_failure_of_a_major_financial_mechanism_or_institution",
               "risk_004_failure_shortfall_of_critical_infrastructure",
               "risk_005_fiscal_crises_in_key_economies",
               "risk_006_high_structural_unemployment_or_underemployment",
               "risk_007_illicit_trade",
               "risk_008_severe_energy_price_shock",
               "risk_009_unmanageable_inflation",
               "risk_010_extreme_weather_events",
               "risk_011_failure_of_climate_change_mitigation_and_adaptation",
               "risk_012_major_biodiversity_loss_and_ecosystem_collapse",
               "risk_013_man_made_environmental_damage_and_disasters",
               "risk_014_failure_of_national_governance",
               "risk_015_failure_of_regional_or_global_governance",
               "risk_016_interstate_conflict_with_regional_consequences",
               "risk_017_large_scale_terrorist_attacks",
               "risk_018_state_collapse_or_crisis",
               "risk_019_weapons_of_mass_destruction",
               "risk_020_failure_of_urban_planning",
               "risk_021_food_crises",
               "risk_022_large_scale_involuntary_migration",
               "risk_023_profound_social_instability",
               "risk_024_rapid_and_massive_spread_of_infectious_diseases",
               "risk_025_water_crises",
               "risk_026_adverse_consequences_of_technological_advances",
               "risk_027_breakdown_of_critical_information_infrastructure_and_networks_and_cyberattacks",
               "risk_028_massive_incident_of_data_fraud_and_theft"
              ]

In [84]:
indexed_risk_tables = zip(range(len(risk_tables)), risk_tables)

In [85]:
indexed_risk_tables[0:10]

[(0, 'risk_001_asset_bubbles_in_a_major_economy'),
 (1, 'risk_002_deflation_in_a_major_economy'),
 (2, 'risk_003_failure_of_a_major_financial_mechanism_or_institution'),
 (3, 'risk_004_failure_shortfall_of_critical_infrastructure'),
 (4, 'risk_005_fiscal_crises_in_key_economies'),
 (5, 'risk_006_high_structural_unemployment_or_underemployment'),
 (6, 'risk_007_illicit_trade'),
 (7, 'risk_008_severe_energy_price_shock'),
 (8, 'risk_009_unmanageable_inflation'),
 (9, 'risk_010_extreme_weather_events')]

### These functions obtain the Entities for a Global Risk, that is, the Persons and Organizations associated to the risk. More precisely, we compute the union of fields V2Persons and V2Organizations. 

In [20]:
def query(from_year, bigquery_full_table_name):
    q = """
    SELECT entities FROM
    (
  (
  SELECT
        LOWER(
          REGEXP_REPLACE(
            ARRAY_TO_STRING(SPLIT(V2Persons,';') ,' ', ''),
            r',.*', '')
            ) as entities
            , count(*) as frequency
    FROM 
      `%s`
    WHERE 
      DATE > %s
    GROUP BY entities
    HAVING frequency >= 100
  )  
  UNION ALL
  (
    SELECT
        LOWER(
          REGEXP_REPLACE(
            ARRAY_TO_STRING(SPLIT(V2Organizations ,';') ,' ', ''),
            r',.*', '')
            ) as entities
            , count(*) as frequency
    FROM 
      `%s`
    WHERE 
      DATE > 2017
    GROUP BY entities
    HAVING frequency >= 100
    )
  )
WHERE entities IS NOT NULL
GROUP BY entities
""" % (from_year, bigquery_full_table_name, bigquery_full_table_name)
    return q

In [23]:
def risk_entities(q):
    entities = bq.Query(q)
    df = entities.execute(output_options=bq.QueryOutput.dataframe()).result()
    return df

---

**The queries for the entities from articles associated to the global risks during 2017:**

In [21]:
queries = map(lambda table_name: query(from_year='2017', table_name), risk_tables)

**The entities as dataframes:**

In [24]:
entity_dfs = map(risk_entities, queries)

**converting the dataframes to sets**

In [25]:
entity_sets = map(lambda df: set(df['entities']), entity_dfs)

In [26]:
[len(x) for x in entity_sets]

[27249,
 25186,
 233,
 4985,
 24984,
 107575,
 86519,
 54398,
 106823,
 50273,
 63473,
 48373,
 886,
 115834,
 84281,
 189840,
 9330,
 40,
 440,
 8904,
 47152,
 87385,
 60721,
 79845,
 73000,
 4629,
 7546,
 2713]

In [38]:
len(entity_sets[0].intersection(entity_sets[2]))

143

**Function to compute the Jackard coefficient [https://en.wikipedia.org/wiki/Jaccard_index]:**

In [62]:
def jackard((a_idx, b_idx), entity_sets):
    a = entity_sets[a_idx]
    b = entity_sets[b_idx]
    return float(len(a.intersection(b)))/float(len(a.union(b)))

---

### Cmputing the interconnection weight between pairs of global risks:

In [72]:
range_entities = range(len(entity_sets))

In [79]:
len(range_entities)

28

In [76]:
pairs = []
for i in range_entities:
    for j in range_entities:
        if i < j:
            pairs.append((i, j))

In [78]:
# number of pairs: (len(range_entities) * (len(range_entities) - 1)) / 2 = 28 * 27 / 2 = 378
len(pairs)

378

In [80]:
pairwise_weights = map(lambda pair: (pair, jackard(pair, entity_sets)), pairs)

In [81]:
pairwise_weights[0:10]

[((0, 1), 0.9162738003873844),
 ((0, 2), 0.005230622919638612),
 ((0, 3), 0.06812910066936179),
 ((0, 4), 0.9168776835847187),
 ((0, 5), 0.1947504142777389),
 ((0, 6), 0.228304290557319),
 ((0, 7), 0.2639637129234008),
 ((0, 8), 0.22228097365302216),
 ((0, 9), 0.21114878060212164),
 ((0, 10), 0.2056880855870822)]

### The necessary output is then saved to the following files:

* entity_sets.txt
* indexed_risk_tables.txt
* pairwise_weights.txt

### availabe at:

https://github.com/libre-ai/minerva/tree/master/data